<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=198646819" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json as js
import subprocess
import logging
import time
import shutil

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2019-01-30 00:00:00,WBTC-USD,3494.277588,3496.455322,3502.762939,3477.248047,1475.0,3496.455322
1,2019-01-31 00:00:00,WBTC-USD,3496.839844,3466.506592,3535.421143,3435.290527,19289.0,3466.506592
2,2019-02-01 00:00:00,WBTC-USD,3469.393066,3480.772217,3517.770264,3422.825684,13640.0,3480.772217
3,2019-02-02 00:00:00,WBTC-USD,3476.335205,3542.095459,3546.404297,3459.112793,1895.0,3542.095459
4,2019-02-03 00:00:00,WBTC-USD,3540.562744,3477.865967,3544.666992,3436.177734,6525.0,3477.865967
...,...,...,...,...,...,...,...,...
2139440,2024-09-22 00:00:00,FIL-USD,3.869217,3.727432,3.874615,3.614296,87602238.0,3.727432
2139441,2024-09-23 00:00:00,FIL-USD,3.727432,3.761462,3.807903,3.676438,107662421.0,3.761462
2139442,2024-09-24 00:00:00,FIL-USD,3.761458,3.859548,3.876596,3.701883,119241729.0,3.859548
2139443,2024-09-25 00:00:00,FIL-USD,3.859548,3.942084,4.042198,3.854716,169053849.0,3.942084


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = f'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'
link

'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(20)):
    url = f'https://finance.yahoo.com/markets/crypto/all/?start={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 20/20 [00:08<00:00,  2.33it/s]


In [6]:
symbol_list = []

for r in tqdm(resp_main_list):
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

100%|██████████| 20/20 [00:13<00:00,  1.50it/s]

2520


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [7]:
'''

llink1 = https://query1.finance.yahoo.com/v8/finance/chart/GLM-USD?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1=1510185600&period2=1725897789&symbol=GLM-USD&userYfid=true&lang=en-US&region=US

here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))

period2 = current_timestamp  

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1420070400&period2=1727527324


# create stocks directory

In [8]:
os.mkdir('crypto_data')

# Download all files

In [9]:
def return_timestamp(unix_timestamps, date_format='%Y-%m-%d %H:%M:%S'):
    formatted_dates = []
    for ut in unix_timestamps:
        date = datetime.fromtimestamp(ut)
        formatted_date = date.strftime(date_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

print(return_timestamp([1725898133]))

['2024-09-09 16:08:53']


In [10]:
for stock in tqdm(symbol_list):
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{stock}?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&symbol={stock}&userYfid=true&lang=en-US&region=US'
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.json", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.json")
    else:
#         print(f"Failed to download file : {stock}.json")
        pass
#     time.sleep(3)

100%|██████████| 2520/2520 [04:00<00:00, 10.48it/s]


In [11]:
symbol_list_dir = os.listdir('/kaggle/working/crypto_data/')
len(symbol_list_dir)

2019

# Read a sample

In [12]:
jsonf = pd.read_json('/kaggle/working/crypto_data/BTC-USD.json')
jsondict = jsonf.to_dict()
timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
symbol = list(sym for s in range(len(timestamp)))
low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
dates = return_timestamp(timestamp)
tempdf = pd.DataFrame(
    {
    'dates':dates,
    'symbol':symbol,
    'open':open_,
    'close':close,
    'high':high,
    'low':low,
    'volume':volume,
    'adj_close':adj_close
    }
)
tempdf

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-01 00:00:00,BTC-USD,320.434998,314.248993,320.434998,314.002991,8036550,314.248993
1,2015-01-02 00:00:00,BTC-USD,314.079010,315.032013,315.838989,313.565002,7860650,315.032013
2,2015-01-03 00:00:00,BTC-USD,314.846008,281.082001,315.149994,281.082001,33054400,281.082001
3,2015-01-04 00:00:00,BTC-USD,281.145996,264.195007,287.230011,257.612000,55629100,264.195007
4,2015-01-05 00:00:00,BTC-USD,265.084015,274.473999,278.341003,265.084015,43962800,274.473999
...,...,...,...,...,...,...,...,...
3554,2024-09-24 00:00:00,BTC-USD,63326.839844,64301.968750,64695.214844,62737.417969,29938335243,64301.968750
3555,2024-09-25 00:00:00,BTC-USD,64302.589844,63143.144531,64804.503906,62945.375000,25078377700,63143.144531
3556,2024-09-26 00:00:00,BTC-USD,63138.546875,65181.019531,65790.796875,62669.269531,36873129847,65181.019531
3557,2024-09-27 00:00:00,BTC-USD,65180.664062,65790.664062,66480.695312,64852.992188,32058813449,65790.664062


# Work on dataset

In [13]:
megadf = pd.DataFrame()

In [14]:
for jso in tqdm(symbol_list_dir):
    jsonf = pd.read_json(f'/kaggle/working/crypto_data/{jso}')
    jsondict = jsonf.to_dict()
    
    # Safely get the timestamp and check if it's None
    timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
    if timestamp is None:
        print(f"Missing timestamp for {jso}. Skipping this symbol.")
        continue  # Skip this iteration if timestamp is None
    
    # Proceed with the rest of the code after ensuring timestamp is not None
    low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
    sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
    
    # Create a list of symbols based on the length of timestamp
    symbol = list(sym for s in range(len(timestamp)))
    
    high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
    volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
    open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
    close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
    adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
    
    # Convert timestamp to dates
    dates = return_timestamp(timestamp)
    
    # Create a DataFrame
    tempdf = pd.DataFrame(
    {
        'dates': dates,
        'symbol': symbol,
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': volume,
        'adj_close': adj_close
    })
    
    # Concatenate the new data to the main DataFrame
    megadf = pd.concat([megadf, tempdf], ignore_index=True)


  8%|▊         | 156/2019 [00:03<00:41, 44.70it/s]

Missing timestamp for MOTH-USD.json. Skipping this symbol.


 15%|█▌        | 311/2019 [00:07<00:53, 32.19it/s]

Missing timestamp for Nasdaq.json. Skipping this symbol.


 20%|█▉        | 398/2019 [00:10<00:58, 27.71it/s]

...

 86%|████████▌ | 1738/2019 [01:57<00:24, 11.49it/s]

Missing timestamp for MOBY32259-USD.json. Skipping this symbol.


 93%|█████████▎| 1869/2019 [02:12<00:15,  9.44it/s]

Missing timestamp for DOAI-USD.json. Skipping this symbol.


100%|██████████| 2019/2019 [02:34<00:00, 13.08it/s]


In [15]:
megadf['symbol'].unique()

array(['INF-USD', 'KWD-USD', 'CTG22540-USD', ..., 'UNO8875-USD',
       'WOM-USD', 'ARDR-USD'], dtype=object)

In [16]:
first_date = megadf['dates'][0]
last_date = megadf['dates'].iloc[-1]

In [17]:
first_date,last_date

('2021-12-09 00:00:00', '2024-09-28 12:40:00')

In [18]:
megadf.dtypes

dates         object
symbol        object
open         float64
close        float64
high         float64
low          float64
volume       float64
adj_close    float64
dtype: object

In [19]:
megadf['dates'] = pd.to_datetime(megadf['dates'], format="mixed")

In [20]:
megadf.dtypes

dates        datetime64[ns]
symbol               object
open                float64
close               float64
high                float64
low                 float64
volume              float64
adj_close           float64
dtype: object

In [21]:
megadf

,dates,symbol,open,close,high,low,volume,adj_close
0,2021-12-09 00:00:00,INF-USD,2.742490,2.470740,2.764911,2.456358,45693.0,2.470740
1,2021-12-10 00:00:00,INF-USD,2.466090,2.343705,2.533977,2.343705,46279.0,2.343705
2,2021-12-11 00:00:00,INF-USD,2.340463,2.362171,2.394707,2.297809,5933.0,2.362171
3,2021-12-12 00:00:00,INF-USD,2.361641,2.414813,2.425249,2.347847,3228.0,2.414813
4,2021-12-13 00:00:00,INF-USD,2.413945,2.252163,2.422558,2.163526,25244.0,2.252163
...,...,...,...,...,...,...,...,...
2170616,2024-09-24 00:00:00,ARDR-USD,0.083017,0.083845,0.087126,0.079531,18864958.0,0.083845
2170617,2024-09-25 00:00:00,ARDR-USD,0.083845,0.080216,0.083994,0.079910,9958102.0,0.080216
2170618,2024-09-26 00:00:00,ARDR-USD,0.080216,0.080478,0.080888,0.077849,8847827.0,0.080478
2170619,2024-09-27 00:00:00,ARDR-USD,0.080478,0.081614,0.081748,0.079014,8500022.0,0.081614


In [22]:
os.mkdir('dataset')

In [23]:
megadf.to_csv(f'dataset/1000_cryptos.csv',index = False)

# final dataset

In [24]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [25]:
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2021-12-09 00:00:00,INF-USD,2.742490,2.470740,2.764911,2.456358,45693.0,2.470740
1,2021-12-10 00:00:00,INF-USD,2.466090,2.343705,2.533977,2.343705,46279.0,2.343705
2,2021-12-11 00:00:00,INF-USD,2.340463,2.362171,2.394707,2.297809,5933.0,2.362171
3,2021-12-12 00:00:00,INF-USD,2.361641,2.414813,2.425249,2.347847,3228.0,2.414813
4,2021-12-13 00:00:00,INF-USD,2.413945,2.252163,2.422558,2.163526,25244.0,2.252163
...,...,...,...,...,...,...,...,...
2170616,2024-09-24 00:00:00,ARDR-USD,0.083017,0.083845,0.087126,0.079531,18864958.0,0.083845
2170617,2024-09-25 00:00:00,ARDR-USD,0.083845,0.080216,0.083994,0.079910,9958102.0,0.080216
2170618,2024-09-26 00:00:00,ARDR-USD,0.080216,0.080478,0.080888,0.077849,8847827.0,0.080478
2170619,2024-09-27 00:00:00,ARDR-USD,0.080478,0.081614,0.081748,0.079014,8500022.0,0.081614


# Upload Dataset

# User secrets

In [26]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [27]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    js.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [28]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 272M/272M [00:04<00:00, 58.9MB/s]


Upload successful: 1000_cryptos.csv (272MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


In [29]:
shutil.rmtree('/kaggle/working/crypto_data')
shutil.rmtree('/kaggle/working/dataset')

# Thank you , and I will see you in next notebook :)